In [1]:
import json

# Define the notebook structure
notebook_content = {
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Project: The Virtual Power Plant\n",
    "## Valuing Data Center Flexibility in ERCOT's Future Grid\n",
    "\n",
    "**Course Project:** MAE/ENE 539  \n",
    "**Model:** Linear Capacity Expansion with Demand Side Flexibility\n",
    "\n",
    "### Overview\n",
    "This notebook investigates the impact of adding a large (1 GW), flexible Data Center load to the ERCOT grid. Unlike traditional demand, this load is modeled as a \"Virtual Power Plant\"—it acts as a firm load most of the time but can \"dispatch\" (shut down) its consumption when grid prices exceed its opportunity cost (Strike Price).\n",
    "\n",
    "### Scenarios to Test\n",
    "1.  **Baseline (Firm Load):** The Data Center must run 24/7 (Strike Price = $9,000/MWh).\n",
    "2.  **Flexible Load (Virtual Battery):** The Data Center allows interruption for a fee (Strike Price = $200 - $1,000/MWh).\n",
    "3.  **Locational Value:** Siting the Data Center in Zone 1 (West Texas Wind) vs. Zone 2 (Urban Demand Center)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "using JuMP\n",
    "using HiGHS\n",
    "using DataFrames, CSV\n",
    "using Statistics"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load Data (16-Week Resolution)\n",
    "We use the higher-resolution 16-week dataset to capture seasonal volatility."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Define path to the 16-week dataset\n",
    "inputs_path = \"ercot_brownfield_expansion/16_weeks/\"\n",
    "\n",
    "# LOAD GENERATORS & STORAGE\n",
    "generators = DataFrame(CSV.File(joinpath(inputs_path, \"Generators_data.csv\")))\n",
    "# Filter columns to keep model lightweight\n",
    "generators = select(generators, :R_ID, :Resource, :zone, :THERM, :DISP, :NDISP, :STOR, :HYDRO, :RPS, :CES,\n",
    "                    :Commit, :Existing_Cap_MW, :Existing_Cap_MWh, :Cap_size, :New_Build, :Max_Cap_MW,\n",
    "                    :Inv_cost_per_MWyr, :Fixed_OM_cost_per_MWyr, :Inv_cost_per_MWhyr, :Fixed_OM_cost_per_MWhyr,\n",
    "                    :Var_OM_cost_per_MWh, :Start_cost_per_MW, :Start_fuel_MMBTU_per_MW, :Heat_rate_MMBTU_per_MWh, :Fuel,\n",
    "                    :Min_power, :Ramp_Up_percentage, :Ramp_Dn_percentage, :Up_time, :Down_time,\n",
    "                    :Eff_up, :Eff_down);\n",
    "\n",
    "# LOAD DEMAND & NETWORK\n",
    "demand_inputs = DataFrame(CSV.File(joinpath(inputs_path, \"Load_data.csv\")))\n",
    "variability = DataFrame(CSV.File(joinpath(inputs_path, \"Generators_variability.csv\")))\n",
    "variability = variability[:,2:ncol(variability)] # Drop index column\n",
    "fuels = DataFrame(CSV.File(joinpath(inputs_path, \"Fuels_data.csv\")))\n",
    "network = DataFrame(CSV.File(joinpath(inputs_path, \"Network.csv\")))\n",
    "\n",
    "println(\"Data Loaded Successfully.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. SCENARIO SETUP: The \"Virtual Power Plant\"\n",
    "**Instructions:** Modify the variables below to test different scenarios.\n",
    "* To test **Firm Load**, set `interruption_strike_price = 9000` (VOLL).\n",
    "* To test **Flexible Load**, set `interruption_strike_price = 500`."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# === USER INPUTS ===\n",
    "data_center_mw = 1000.0         # Size of the new load\n",
    "data_center_zone = 1            # Zone 1 (West TX) or Zone 2 (East TX)\n",
    "interruption_strike_price = 300.0 # $/MWh (The \"cost\" to shut down)\n",
    "\n",
    "# === MODELING LOGIC ===\n",
    "\n",
    "# 1. Add Flat Load to Demand\n",
    "demand = select(demand_inputs, :Load_MW_z1, :Load_MW_z2, :Load_MW_z3)\n",
    "if data_center_zone == 1\n",
    "    demand.Load_MW_z1 .+= data_center_mw\n",
    "elseif data_center_zone == 2\n",
    "    demand.Load_MW_z2 .+= data_center_mw\n",
    "elseif data_center_zone == 3\n",
    "    demand.Load_MW_z3 .+= data_center_mw\n",
    "end\n",
    "println(\"Added $(data_center_mw) MW flat load to Zone $(data_center_zone).\")\n",
    "\n",
    "# 2. Create \"Virtual Generator\" (Demand Response Resource)\n",
    "# This generator produces \"negawatts\" (cancels the load) at the strike price.\n",
    "new_dr = copy(generators[1, :]) # Template row\n",
    "\n",
    "new_dr.R_ID = \"DataCenter_DR\"\n",
    "new_dr.Resource = \"Demand_Response\"\n",
    "new_dr.zone = data_center_zone\n",
    "new_dr.THERM = 0; new_dr.DISP = 1; new_dr.STOR = 0; new_dr.HYDRO = 0\n",
    "new_dr.Existing_Cap_MW = data_center_mw # Capacity matches the load size\n",
    "new_dr.Max_Cap_MW = data_center_mw      # Cannot expand beyond the load size\n",
    "new_dr.New_Build = 0                    # It's an existing asset (the contract)\n",
    "new_dr.Var_OM_cost_per_MWh = interruption_strike_price\n",
    "new_dr.Fuel = \"DR_Virtual\"\n",
    "new_dr.Heat_rate_MMBTU_per_MWh = 0.0\n",
    "new_dr.Min_power = 0.0\n",
    "new_dr.Ramp_Up_percentage = 1.0 # Instant response\n",
    "new_dr.Ramp_Dn_percentage = 1.0\n",
    "\n",
    "# Clean up other cost columns to be zero\n",
    "new_dr.Inv_cost_per_MWyr = 0.0\n",
    "new_dr.Fixed_OM_cost_per_MWyr = 0.0\n",
    "new_dr.Start_cost_per_MW = 0.0\n",
    "\n",
    "# Add to generators list\n",
    "push!(generators, new_dr)\n",
    "G = generators.R_ID\n",
    "\n",
    "println(\"Created 'DataCenter_DR' resource with strike price $$(interruption_strike_price)/MWh.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Pre-Processing (Calculations)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Process Time/Sample Weights\n",
    "hours_per_period = convert(Int64, demand_inputs.Hours_per_period[1])\n",
    "P = convert(Array{Int64}, 1:demand_inputs.Subperiods[1])\n",
    "W = convert(Array{Int64}, collect(skipmissing(demand_inputs.Sub_Weights)))\n",
    "T = convert(Array{Int64}, demand_inputs.Time_index)\n",
    "sample_weight = zeros(Float64, size(T,1))\n",
    "t=1\n",
    "for p in P\n",
    "    for h in 1:hours_per_period\n",
    "        sample_weight[t] = W[p]/hours_per_period\n",
    "        t=t+1\n",
    "    end\n",
    "end\n",
    "\n",
    "# Process Variable Costs & CO2\n",
    "generators.Var_Cost = zeros(Float64, size(G,1))\n",
    "for g in 1:nrow(generators)\n",
    "    # If it is the DR resource, cost is just the OM cost (Strike Price)\n",
    "    if generators.Fuel[g] == \"DR_Virtual\"\n",
    "        generators.Var_Cost[g] = generators.Var_OM_cost_per_MWh[g]\n",
    "    else\n",
    "        # Normal Generators: Fuel * HeatRate + VOM\n",
    "        fuel_cost = fuels[fuels.Fuel.==generators.Fuel[g],:Cost_per_MMBtu][1]\n",
    "        generators.Var_Cost[g] = generators.Var_OM_cost_per_MWh[g] + \n",
    "                                 fuel_cost * generators.Heat_rate_MMBTU_per_MWh[g]\n",
    "    end\n",
    "end\n",
    "\n",
    "# Define Sets\n",
    "S = convert(Array{Int64}, collect(skipmissing(demand_inputs.Demand_segment)))\n",
    "Z = convert(Array{Int64}, 1:3)\n",
    "zones = collect(skipmissing(network.Network_zones))\n",
    "lines = select(network[1:2,:], :Network_lines, :z1, :z2, :z3, :Line_Max_Flow_MW, \n",
    "               :Line_Reinforcement_Cost_per_MW_yr)\n",
    "lines.Line_Fixed_Cost_per_MW_yr = lines.Line_Reinforcement_Cost_per_MW_yr./20\n",
    "L = convert(Array{Int64}, lines.Network_lines)\n",
    "\n",
    "# NSE Data (Value of Lost Load)\n",
    "VOLL = demand_inputs.Voll[1]\n",
    "nse = DataFrame(Segment=S, \n",
    "                NSE_Cost = VOLL.*collect(skipmissing(demand_inputs.Cost_of_demand_curtailment_perMW)),\n",
    "                NSE_Max = collect(skipmissing(demand_inputs.Max_demand_curtailment)))\n",
    "\n",
    "# Subsets\n",
    "STOR = intersect(generators.R_ID[generators.STOR.>=1], G)\n",
    "NEW = intersect(generators.R_ID[generators.New_Build.==1], G)\n",
    "OLD = intersect(generators.R_ID[.!(generators.New_Build.==1)], G)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Optimization Model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "Expansion_Model = Model(HiGHS.Optimizer)\n",
    "\n",
    "# --- VARIABLES ---\n",
    "@variables(Expansion_Model, begin\n",
    "    vCAP[g in G] >= 0\n",
    "    vNEW_CAP[g in NEW] >= 0\n",
    "    vRET_CAP[g in OLD] >= 0\n",
    "    vE_CAP[g in STOR] >= 0\n",
    "    vNEW_E_CAP[g in intersect(STOR, NEW)] >= 0\n",
    "    vRET_E_CAP[g in intersect(STOR, OLD)] >= 0\n",
    "    vT_CAP[l in L] >= 0\n",
    "    vNEW_T_CAP[l in L] >= 0\n",
    "    vGEN[T,G] >= 0\n",
    "    vCHARGE[T,STOR] >= 0\n",
    "    vSOC[T,STOR] >= 0\n",
    "    vNSE[T,S,Z] >= 0\n",
    "    vFLOW[T,L]\n",
    "end)\n",
    "\n",
    "# --- CONSTRAINTS ---\n",
    "\n",
    "# 1. Demand Balance (Includes Data Center Load)\n",
    "@constraint(Expansion_Model, cDemandBalance[t in T, z in Z],\n",
    "    sum(vGEN[t,g] for g in intersect(generators[generators.zone.==z,:R_ID],G)) +\n",
    "    sum(vNSE[t,s,z] for s in S) -\n",
    "    sum(vCHARGE[t,g] for g in intersect(generators[generators.zone.==z,:R_ID],STOR)) -\n",
    "    demand[t,z] - \n",
    "    sum(lines[l,Symbol(string(\"z\",z))] * vFLOW[t,l] for l in L) == 0\n",
    ")\n",
    "\n",
    "# 2. Generator Constraints (Capacity & Operations)\n",
    "for g in NEW[generators[NEW,:Max_Cap_MW].>0]\n",
    "    set_upper_bound(vNEW_CAP[g], generators.Max_Cap_MW[g])\n",
    "end\n",
    "\n",
    "# Operational Limits\n",
    "@constraints(Expansion_Model, begin\n",
    "    cMaxPower[t in T, g in G], vGEN[t,g] <= variability[t,g]*vCAP[g]\n",
    "    cCapOld[g in OLD], vCAP[g] == generators.Existing_Cap_MW[g] - vRET_CAP[g]\n",
    "    cCapNew[g in NEW], vCAP[g] == vNEW_CAP[g]\n",
    "end)\n",
    "\n",
    "# Storage Constraints\n",
    "@constraints(Expansion_Model, begin\n",
    "    cMaxCharge[t in T, g in STOR], vCHARGE[t,g] <= vCAP[g]\n",
    "    cMaxSOC[t in T, g in STOR], vSOC[t,g] <= vE_CAP[g]\n",
    "    cCapEnergyOld[g in intersect(STOR, OLD)], vE_CAP[g] == generators.Existing_Cap_MWh[g] - vRET_E_CAP[g]\n",
    "    cCapEnergyNew[g in intersect(STOR, NEW)], vE_CAP[g] == vNEW_E_CAP[g]\n",
    "end)\n",
    "\n",
    "# Transmission Constraints\n",
    "@constraints(Expansion_Model, begin\n",
    "    cTransCap[l in L], vT_CAP[l] == lines.Line_Max_Flow_MW[l] + vNEW_T_CAP[l]\n",
    "    cMaxFlow[t in T, l in L], vFLOW[t,l] <= vT_CAP[l]\n",
    "    cMinFlow[t in T, l in L], vFLOW[t,l] >= -vT_CAP[l]\n",
    "end)\n",
    "\n",
    "# Time Coupling (Ramping & Storage)\n",
    "STARTS = 1:hours_per_period:maximum(T)\n",
    "INTERIORS = setdiff(T,STARTS)\n",
    "\n",
    "@constraints(Expansion_Model, begin\n",
    "    # Ramping (Simplified)\n",
    "    cRampUp[t in INTERIORS, g in G], vGEN[t,g] - vGEN[t-1,g] <= generators.Ramp_Up_percentage[g]*vCAP[g]\n",
    "    \n",
    "    # Storage Balance\n",
    "    cSOC[t in INTERIORS, g in STOR], \n",
    "        vSOC[t,g] == vSOC[t-1,g] + generators.Eff_up[g]*vCHARGE[t,g] - vGEN[t,g]/generators.Eff_down[g]\n",
    "end)\n",
    "\n",
    "# --- OBJECTIVE ---\n",
    "@objective(Expansion_Model, Min,\n",
    "    sum(generators.Fixed_OM_cost_per_MWyr[g]*vCAP[g] for g in G) +\n",
    "    sum(generators.Inv_cost_per_MWyr[g]*vNEW_CAP[g] for g in NEW) +\n",
    "    sum(generators.Fixed_OM_cost_per_MWhyr[g]*vE_CAP[g] for g in STOR) +\n",
    "    sum(generators.Inv_cost_per_MWhyr[g]*vNEW_E_CAP[g] for g in intersect(STOR, NEW)) +\n",
    "    sum(lines.Line_Fixed_Cost_per_MW_yr[l]*vT_CAP[l] + lines.Line_Reinforcement_Cost_per_MW_yr[l]*vNEW_T_CAP[l] for l in L) +\n",
    "    sum(sample_weight[t]*generators.Var_Cost[g]*vGEN[t,g] for t in T, g in G) +\n",
    "    sum(sample_weight[t]*nse.NSE_Cost[s]*vNSE[t,s,z] for t in T, s in S, z in Z)\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Solve and Analyze"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": [
    "println(\"Solving Model... This may take a few minutes.\")\n",
    "@time optimize!(Expansion_Model)\n",
    "\n",
    "# Analyze Data Center Flexibility Usage\n",
    "dc_gen = value.(vGEN)[:, \"DataCenter_DR\"]\n",
    "total_dc_curtailment_MWh = sum(sample_weight .* dc_gen)\n",
    "total_dc_hours_curtailed = count(x -> x > 1.0, dc_gen)\n",
    "\n",
    "println(\"--- DATA CENTER RESULTS ---\")\n",
    "println(\"Scenario Strike Price: $$(interruption_strike_price) / MWh\")\n",
    "println(\"Total Load Curtailed (Virtual Generation): $(round(total_dc_curtailment_MWh, digits=0)) MWh\")\n",
    "println(\"Number of Hours Curtailed: $total_dc_hours_curtailed hours\")\n",
    "\n",
    "# Quick Look at Capacity Expansion\n",
    "new_builds = DataFrame(Resource = generators.Resource[NEW], \n",
    "                       New_MW = value.(vNEW_CAP).data)\n",
    "filter!(row -> row.New_MW > 1.0, new_builds)\n",
    "println(\"\\n--- NEW CAPACITY BUILT ---\")\n",
    "show(new_builds, allrows=true)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Julia 1.9",
   "language": "julia",
   "name": "julia-1.9"
  },
  "language_info": {
   "file_extension": ".jl",
   "mimetype": "application/julia",
   "name": "julia",
   "version": "1.9.3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}

# Create the file
with open('DataCenter_Project.ipynb', 'w', encoding='utf-8') as f:
    json.dump(notebook_content, f, indent=1)

print("Success! File 'DataCenter_Project.ipynb' has been created in your current directory.")

Success! File 'DataCenter_Project.ipynb' has been created in your current directory.
